In [20]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from keras.models import load_model
import numpy as np 
import pandas as pd 
import os
import glob
import cv2
import random
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import regularizers
from keras.callbacks import CSVLogger
#from livelossplot import PlotLossesKeras
import os
import numpy as np
#from imgaug import augmenters as iaa
#import cv2
from keras.layers.normalization import BatchNormalization
#import seaborn as sns
import pandas as pd 
from keras import initializers
from keras import optimizers
import keras.backend as K
import tensorflow as tf
import operator

In [21]:
print(os.listdir("../input/validation-data/val"))

['all', 'hem']


In [22]:
def crop_center(img, bounding):
    start = tuple(map(lambda a, da: a//2-da//2, img.shape, bounding))
    end = tuple(map(operator.add, start, bounding))
    slices = tuple(map(slice, start, end))
    return img[slices]

In [23]:
PATH = "../input/validation-data/val"

In [24]:
# all image path of all and hem
all_para=glob.glob(os.path.join(PATH,'all','*.bmp'))
paths_hem=glob.glob(os.path.join(PATH,'hem','*.bmp'))
len(all_para), len(paths_hem)

(1219, 648)

In [25]:
# helper to get images
def get_data(path, resize_dim=210):
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=crop_center(img,(210,210))
    #img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 
    #img_rgb = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
    
    img_rz=cv2.resize(img,(resize_dim,resize_dim),interpolation=cv2.INTER_AREA)
    
    return img_rz

In [26]:
parasite_images=np.array([get_data(path) for path in all_para])
parasite_images.shape

(1219, 210, 210, 3)

In [27]:
uninf_images=np.array([get_data(path) for path in paths_hem])
uninf_images.shape

(648, 210, 210, 3)

In [28]:
para_label = np.array([0 for _ in range(len(parasite_images))])
uninf_label = np.array([1 for _ in range(len(uninf_images))])

para_label.shape, uninf_label.shape

((1219,), (648,))

In [29]:
x_all = np.concatenate((parasite_images, uninf_images), axis=0)
y_all = np.concatenate((para_label, uninf_label), axis=0)
print(x_all.shape, y_all.shape)

(1867, 210, 210, 3) (1867,)


# 5x Aug Adam_baseline

In [30]:
model = None
model = load_model("../input/modelsforleukemia/Models for 5x Aug/adam_baseline_vgg.h5", compile = False)
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 210, 210, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 210, 210, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 210, 210, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 105, 105, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 105, 105, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 105, 105, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 52, 52, 128)       0     

In [31]:
# Make predictions using trained model
y_pred = model.predict(x_all, verbose=1)
print("Predictions: ", y_pred.shape)

59/59 [==============================] - 3s 56ms/step
Predictions:  (1867, 1)


In [32]:
y_pred_flat = []
for pred in y_pred:
   if pred > 0.5:
       y_pred_flat.append(1)
   else:
       y_pred_flat.append(0)
y_pred_flat = np.array(y_pred_flat)

In [33]:
from sklearn.metrics import confusion_matrix, classification_report

# Classification report

confusion_mtx = confusion_matrix(y_all, y_pred_flat) 
print(confusion_mtx)
target_names = ['0', '1']
print(classification_report(y_all, y_pred_flat, target_names=target_names, digits=4))

[[   1 1218]
 [   4  644]]
              precision    recall  f1-score   support

           0     0.2000    0.0008    0.0016      1219
           1     0.3459    0.9938    0.5131       648

    accuracy                         0.3455      1867
   macro avg     0.2729    0.4973    0.2574      1867
weighted avg     0.2506    0.3455    0.1792      1867



# 5x Aug SGD